In [1]:
import pandas as pd

In [6]:
dfg_attributes = pd.read_csv("../Data/dataframes_to_repo/EDA_df_to_repo/dfg_attributes.csv")
dfg_attributes.head()

C:\Users\dmon2\AppData\Local\Temp\ipykernel_5028\4292586480.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  dfg_attributes = pd.read_csv("../Data/dataframes_to_repo/EDA_df_to_repo/dfg_attributes.csv")


,Unnamed: 0,gmap_id,Accessibility,Activities,Amenities
0,2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,['Wheelchair accessible entrance'],NaN,['Good for kids']
1,6,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,['Wheelchair accessible entrance'],NaN,['Good for kids']
2,8,0x87ec235c54d25b31:0x3b75fb5facc602f,NaN,NaN,NaN
3,68,0x89c6c89efcaed69d:0xded973f6033e7dba,NaN,NaN,['Good for kids']
4,75,0x87fd0e70c5f5d87b:0xdf340eeb75040ef3,NaN,NaN,['Restroom']


In [12]:
def combine_columns(row):
    combined = []
    for col in ['Accessibility', 'Activities', 'Amenities']:
        if pd.notna(row[col]):
            combined.extend(row[col])  # Usar el método extend para combinar listas sin descomponer los elementos
    return combined

dfg_attributes['Combined'] = dfg_attributes.apply(combine_columns, axis=1)

In [13]:
dfg_attributes.head()

,Unnamed: 0,gmap_id,Accessibility,Activities,Amenities,Combined
0,2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,['Wheelchair accessible entrance'],NaN,['Good for kids'],"[[, ', W, h, e, e, l, c, h, a, i, r, , a, c, ..."
1,6,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,['Wheelchair accessible entrance'],NaN,['Good for kids'],"[[, ', W, h, e, e, l, c, h, a, i, r, , a, c, ..."
2,8,0x87ec235c54d25b31:0x3b75fb5facc602f,NaN,NaN,NaN,[]
3,68,0x89c6c89efcaed69d:0xded973f6033e7dba,NaN,NaN,['Good for kids'],"[[, ', G, o, o, d, , f, o, r, , k, i, d, s, ..."
4,75,0x87fd0e70c5f5d87b:0xdf340eeb75040ef3,NaN,NaN,['Restroom'],"[[, ', R, e, s, t, r, o, o, m, ', ]]"


In [5]:
# Función para procesar los valores de la columna Accessibility
def process_value(value):
    # Verificar si el valor es nulo
    if pd.isnull(value):
        return None
    # Verificar si el valor es una lista
    if isinstance(value, list):
        return value
    # Dividir la cadena por líneas y eliminar elementos vacíos
    return value.splitlines() if isinstance(value, str) else None

# Aplicar la función a la columna Accessibility
dfg_attributes['Accessibility'] = dfg_attributes['Accessibility'].apply(process_value)
dfg_attributes['Accessibility'].head(5)

0    [['Wheelchair accessible entrance']]
1    [['Wheelchair accessible entrance']]
2                                    None
3                                    None
4                                    None
Name: Accessibility, dtype: object

In [7]:
# Dividir y limpiar los valores basados en el espacio en blanco
dfg_attributes['Accessibility'] = dfg_attributes['Accessibility'].apply(lambda x: x.split() if pd.notnull(x) else [])
dfg_attributes['Activities'] = dfg_attributes['Activities'].apply(lambda x: x.split() if pd.notnull(x) else [])
dfg_attributes['Amenities'] = dfg_attributes['Amenities'].apply(lambda x: x.split() if pd.notnull(x) else [])

# Muestra el DataFrame resultante
dfg_attributes.head()

,Unnamed: 0,gmap_id,Accessibility,Activities,Amenities
0,2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,"[['Wheelchair, accessible, entrance']]",[],"[['Good, for, kids']]"
1,6,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,"[['Wheelchair, accessible, entrance']]",[],"[['Good, for, kids']]"
2,8,0x87ec235c54d25b31:0x3b75fb5facc602f,[],[],[]
3,68,0x89c6c89efcaed69d:0xded973f6033e7dba,[],[],"[['Good, for, kids']]"
4,75,0x87fd0e70c5f5d87b:0xdf340eeb75040ef3,[],[],[['Restroom']]


In [32]:
from google.cloud import bigquery

import pandas as pd
import numpy as np

# Autentica tu cuenta de GCP
client = bigquery.Client()

# Se cargan los datos de Big Query
dataset_id = "proyecto-nuevo-423502.Data_Henry_Final"

# Restaurantes
table_id_restaurantes = "proyecto-nuevo-423502.Data_Henry_Final.restaurantes"
table_id_atributos = "proyecto-nuevo-423502.Data_Henry_Final.atributos"

In [17]:
dfgy_rest = pd.read_csv("../Data/dfgy_rest.csv")
dfgy_rest.sample(10)

C:\Users\dmon2\AppData\Local\Temp\ipykernel_5028\1491434318.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  dfgy_rest = pd.read_csv("../Data/dfgy_rest.csv")


,source,site_id,name,state,city,postal_code,price,rating_avg,reviews_count,date_start,year,month,state_city,city_postalcode,state_city_postalcode
35282,yelp,8sJwVOr_dHVrvqOB0L6aoQ,Main Street Pizza,PA,Darby,19023,1.200000,3.0,6,2014-02-25 04:50:43,2014.0,2.0,PA - Darby,Darby - 19023,PA - Darby - 19023
90112,google,0x8702d99c70341e83:0x556734003e68b022,Saeng's Orient,NM,Clovis,88101,1.000000,4.2,108,2012-11-15 20:49:35.235,2012.0,11.0,NM - Clovis,Clovis - 88101,NM - Greenwood - 88101
224864,google,0x89c28da776d62a69:0x775fc1b764652a7a,Fish N' Ting On The Go,NY,Mt Vernon,10552,1.333333,4.0,18,2021-02-10 22:13:42.979,2021.0,2.0,NY - Mt Vernon,Mt Vernon - 10552,NY - Greenwood - 10552
82311,google,0x89c6ceb4e5bda717:0x12e30e6302ad0dc6,Rmac's Publick House,NJ,Oaklyn,8107,1.000000,3.9,178,2016-02-14 17:35:46.015,2016.0,2.0,NJ - Oaklyn,Oaklyn - 8107,NJ - Nashville - 8107
18923,yelp,H_yVbOCHTEocWeXl6UXkIQ,Toto's Pizzeria,PA,Philadelphia,19119,1.500000,3.0,136,2010-08-28 02:01:30,2010.0,8.0,PA - Philadelphia,Philadelphia - 19119,PA - Philadelphia - 19119
72364,google,0x86d671f95dd1061b:0x4ffac31b6fbcd778,Mira Mar,AZ,Tucson,85719,1.416667,4.9,8,NaN,NaN,NaN,AZ - Tucson,Tucson - 85719,AZ - Madeira Beach - 85719
173997,google,0x88fba3fe26454527:0x74e163b77325a3b3,Honey BeeZzz Kitchen,GA,Pooler,31322,1.500000,3.4,28,2018-06-03 06:19:25.315,2018.0,6.0,GA - Pooler,Pooler - 31322,GA - Richmond Heights - 31322
148032,google,0x89c6cf63098cbbed:0x18bea09df8304223,Domino's Pizza,NJ,Brooklawn,8030,1.000000,3.3,28,2021-01-02 15:35:37.762,2021.0,1.0,NJ - Brooklawn,Brooklawn - 8030,NJ - Reno - 8030
184731,google,0x886469d20641891d:0x7dede7255934a433,Domino’s Pizza,TN,Nashville,37214,1.214286,1.0,1,NaN,NaN,NaN,TN - Nashville,Nashville - 37214,TN - Hendersonville - 37214
15798,yelp,Cmo0C0TgMbY0RUlsNR2wRw,Mainland Golf Course,PA,Harleysville,19438,1.000000,3.5,9,2014-10-07 14:04:34,2014.0,10.0,PA - Harleysville,Harleysville - 19438,PA - Harleysville - 19438


In [18]:
dfgy_rest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274060 entries, 0 to 274059
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   source                 274060 non-null  object 
 1   site_id                274060 non-null  object 
 2   name                   274059 non-null  object 
 3   state                  274060 non-null  object 
 4   city                   274060 non-null  object 
 5   postal_code            274060 non-null  int64  
 6   price                  274060 non-null  float64
 7   rating_avg             274060 non-null  float64
 8   reviews_count          274060 non-null  int64  
 9   date_start             182440 non-null  object 
 10  year                   182440 non-null  float64
 11  month                  182440 non-null  float64
 12  state_city             274060 non-null  object 
 13  city_postalcode        274060 non-null  object 
 14  state_city_postalcode  261663 non-nu

In [19]:
## Restaurantes
    # Eliminación de duplicados
dfgy_rest= dfgy_rest.drop_duplicates(subset=['site_id'], keep='first')
    # Procesamiento del tipo de dato.
dfgy_rest[["source",'site_id',"name","state","city","state_city","city_postalcode","state_city_postalcode"]] = dfgy_rest[["source",'site_id',"name","state","city","state_city","city_postalcode","state_city_postalcode"]].astype(str)
dfgy_rest[["postal_code","year","month"]] = dfgy_rest[["postal_code","year","month"]].astype(float).fillna(0).astype(int)

In [25]:
sites_atributos = pd.read_parquet("../Data/dfgy_attributes.parquet")

In [27]:
# Se seleccionan los atributos que van a ser unificados o corregidos a través de un diccionario de mapeo.
reemplazo_atributos = {
    "Accessible entrance":['Wheelchair accessible entrance','Wheelchair-accessible entrance',"WheelchairAccessible"],
    "Accessible elevator":["Wheelchair accessible elevator","Wheelchair-accessible lift"],
    "Accessible seating":['Wheelchair accessible seating','Wheelchair-accessible seating'],
    "Accessible restroom":['Wheelchair accessible restroom','Wheelchair-accessible toilet'],
    "Accessible parking":['Wheelchair accessible parking lot','Wheelchair-accessible car park'],
    "Wifi":['WiFi','Wi-Fi'],
    "Delivery&TakeOut":["RestaurantsDelivery",'RestaurantsTakeOut'],
    "Outdoor Seating":['OutdoorSeating'],
    "Accepts Cards":['BusinessAcceptsCreditCards',"BusinessAcceptsBitcoin"],
    "Parking":["BusinessParking",'BikeParking'],
    "Appointment Only":['ByAppointmentOnly'],
    "Reservations":['RestaurantsReservations'],
    "HappyHour":['HappyHour','BestNights'],
    "BYOB":['BYOB', 'Corkage', 'BYOBCorkage'],
    'Good for kids':['Good for kids','GoodForKids'],
    "Sells Alcohol":['Alcohol','Bar onsite','Bar on site'],
    "Restroom":['Restroom','Gender-neutral restroom','Public restroom','Toilets','Gender-neutral toilets','Public toilet'],
    'Baggage storage':['Baggage storage','CoatCheck']

}

# Se seleccionan los atributos que no proporcionan información útil para el proceso de análisis.
eliminar_atributos = ['Caters','RestaurantsPriceRange2','Ambience','RestaurantsTableService','NoiseLevel',
                      'Music','AcceptsInsurance','AgesAllowed','HairSpecializesIn','High chairs', 'Restaurant','Golf course',
                      'Mechanic','Swimming pool', 'All-inclusive', 'Stadium seating']



In [28]:
# Se invierte el diccionario de mapeo para usarlo en el método "replace".
reverse_mapping = {v: k for k, values in reemplazo_atributos.items() for v in values}

# Se reeemplazan los valores en la columna 'atributos' usando el diccionario invertido
sites_atributos['attributes'] = sites_atributos['attributes'].replace(reverse_mapping)

# Se filtra y eliminan las filas que contienen atributos en eliminar_atributos
sites_atributos = sites_atributos[~sites_atributos['attributes'].str.contains('|'.join(eliminar_atributos))]

sites_atributos.drop_duplicates(keep='first',inplace=True)

In [29]:
sites_atributos["attributes"].unique()

array(['Delivery&TakeOut', 'Outdoor Seating', 'Accepts Cards', 'Parking',
       'Appointment Only', 'Wifi', 'Sells Alcohol', 'Accessible entrance',
       'Good for kids', 'Reservations', 'Baggage storage', 'DogsAllowed',
       'HasTV', 'HappyHour', 'DriveThru', 'GoodForMeal', 'Smoking',
       'GoodForDancing', 'BYOB', 'Open24Hours', 'DietaryRestrictions',
       'Restroom', 'Air conditioning', 'Accessible elevator',
       'Accessible seating', 'Accessible restroom', 'Accessible parking',
       'Assisted listening devices'], dtype=object)

In [33]:
# Se crean dos diccionarios para relacionar a cada uno de los dataframes procesados con su tabla correspondiente en BigQuery

# Relacionamiento del DataFrames al ID de la tabla correspondiente en BigQuery
dataframe_to_table_map = {
    "dataframe_restaurantes": table_id_restaurantes,
    "dataframe_atributos": table_id_atributos,
}

# Obtención de los DataFrames finales
dataframes = {
    "dataframe_restaurantes": dfgy_rest,
    "dataframe_atributos": sites_atributos,
}

In [34]:
# Carga de cada uno de los DataFrame en su tabla correspondiente.

for dataframe_name, table_id in dataframe_to_table_map.items():
    # Obtener el DataFrame desde la variable.
    dataframe = dataframes[dataframe_name]

    # Se configura el trabajo de carga que se utilizará para subir la información a las tablas de BiqQuery.
    job_config = bigquery.LoadJobConfig() # Se define el objeto que permitirá la subida de la información.
    job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE  # Permite sobrescribir la tabla si ya existe.

    # Cargar el DataFrame en la tabla de BigQuery.
    load_job = client.load_table_from_dataframe(dataframe, table_id, job_config=job_config)
    load_job.result()  # Esperar a que el trabajo de carga se complete

    print(f"Successfully loaded {dataframe_name} into table {table_id}")

Successfully loaded dataframe_restaurantes into table proyecto-nuevo-423502.Data_Henry_Final.restaurantes
Successfully loaded dataframe_atributos into table proyecto-nuevo-423502.Data_Henry_Final.atributos
